## 📘 Project Phase: Final Model Expansion

In this notebook, I will build on the previously improved XGBoost model (which achieved an RMSLE of 0.2381) by integrating additional data sources provided in the competition:

- `stores.csv` – store-level metadata (e.g. city, type, cluster)
- `oil.csv` – daily oil prices (macroeconomic signal)
- `transactions.csv` – store-level daily customer volume

### 🛠️ Planned Steps:
1. **Merge external datasets** into the training and test sets.
2. **Engineer new features** from these sources (e.g. oil trends, transaction lags).
3. **Refine the model** by re-tuning hyperparameters using TimeSeriesSplit CV.
4. **Evaluate** the impact of the new data on forecast accuracy (RMSLE).
5. **Re-train on full data** and prepare final Kaggle submission.

The goal is to push the model beyond current performance by capturing more contextual and behavioral signals.

In [292]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

In [293]:
df = pd.read_csv("train.csv", parse_dates=['date'])
df.set_index('date', inplace=True)

holidays = pd.read_csv('holidays_events.csv', parse_dates=['date'])
holidays.set_index('date', inplace=True)

oil_price = pd.read_csv('oil.csv', parse_dates=['date'])
oil_price.set_index('date', inplace=True)

stores = (
    pd.read_csv("stores.csv")
      .rename(columns={"type": "store_type"})
)

transactions = pd.read_csv('transactions.csv', parse_dates=['date'])
transactions.set_index('date', inplace=True)

In [294]:
oil_filled = (
  oil_price
    .asfreq("D")
    .ffill()
    .bfill()
    )

In [295]:
df = (
    df
    .reset_index()
    .merge(
        oil_filled.reset_index(),
        on="date",
        how="left"
    )
    .set_index("date")
)

In [296]:
df = (
    df
    .reset_index()
    .merge(
        holidays.reset_index(),
        on="date",
        how="left"
    )
    .set_index("date")
)

In [297]:
df['is_holiday'] = df['type'].notna().astype(int)
cols = ['type','locale','locale_name','description']
df[cols] = df[cols].fillna('NO_HOLIDAY')
df['transferred'] = df['transferred'].fillna(0).astype(int)

In [298]:
df = (
    df
    .reset_index()
    .merge(
        stores.reset_index(),
        on="store_nbr",
        how="left"
    )
    .set_index("date")
)

In [299]:
transactions.sort_values(['date', 'store_nbr'])

,store_nbr,transactions
date,,
2013-01-01,25,770
2013-01-02,1,2111
2013-01-02,2,2358
2013-01-02,3,3487
2013-01-02,4,1922
...,...,...
2017-08-15,50,2804
2017-08-15,51,1573
2017-08-15,52,2255


In [300]:
df = (
    df.reset_index()
      .merge(
          transactions,
          on=["date", "store_nbr"],
          how="left"
      )
      .set_index("date")
)

In [301]:
df["transactions"] = (
    df.groupby("store_nbr")["transactions"]
      .transform(lambda s: s.ffill().bfill())
)

In [306]:
sample = df.sample(1)
date_, store_, fam_ = sample.index[0], sample["store_nbr"].iloc[0], sample["family"].iloc[0]

print("🔎  sample key:", date_, store_, fam_)
display(df.loc[(date_,), :].query("store_nbr == @store_ and family == @fam_"))

🔎  sample key: 2016-05-15 00:00:00 43 LIQUOR,WINE,BEER


,id,store_nbr,family,sales,onpromotion,dcoilwtico,type,locale,locale_name,description,transferred,is_holiday,index,city,state,store_type,cluster,transactions
date,,,,,,,,,,,,,,,,,,
2016-05-15,2187757,43,"LIQUOR,WINE,BEER",25.0,0,46.22,Event,National,Ecuador,Terremoto Manabi+29,0,1,42,Esmeraldas,Esmeraldas,E,10,1321.0


In [307]:
print("transactions:", transactions.query("date == @date_ and store_nbr == @store_").head())
print("oil:",         oil_filled.loc[date_])
print("holidays:",    holidays.loc[date_])

transactions:             store_nbr  transactions
date                               
2016-05-15         43          1321
oil: dcoilwtico    46.22
Name: 2016-05-15 00:00:00, dtype: float64
holidays: type                         Event
locale                    National
locale_name                Ecuador
description    Terremoto Manabi+29
transferred                  False
Name: 2016-05-15 00:00:00, dtype: object
